<a href="https://colab.research.google.com/github/nedokormysh/GB_PyTorch/blob/lesson9/GB_PyTorch_hw_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.7 MB/s eta 0:00:00


In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
from transformers import pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

import pandas as pd

Загрузка датасета.

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-01-09 10:27:23--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 142.251.18.113, 142.251.18.101, 142.251.18.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.18.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/88bpn4rd3qo2mnva8m1ids9emo3hmrqn/1673259975000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=b472a832-d3d7-4232-9420-15a18851628e [following]
--2023-01-09 10:27:27--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/88bpn4rd3qo2mnva8m1ids9emo3hmrqn/1673259975000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=b472a832-d3d7-4232-9420-15a18851628e
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 108.177.119.132, 2a00

In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [5]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## RuBert-toxic

Возьмём модель sismetanin/rubert-toxic-pikabu-2ch.

In [9]:
# Загрузка модели. Проверка работы модели на токсичной фразе
sentRuBert = pipeline('text-classification', model='sismetanin/rubert-toxic-pikabu-2ch')
sentRuBert("Я убью тебя с ак47")

Downloading:   0%|          | 0.00/920 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.8942258358001709}]

In [10]:
# Загрузка токенайзера
tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])
print(sentRuBert(example_text))

tensor([[   101,  22632,  22574,   2748, 111601,  17510,   3426,    102,      0,
              0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])
[{'label': 'LABEL_0', 'score': 0.9928885102272034}]


In [11]:
tokenizer.ids_to_tokens[22632], tokenizer.ids_to_tokens[111601]

('пример', 'токе')

In [12]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] пример текста для токенизации [SEP] [PAD] [PAD]


In [13]:
# Проверка работы модели на примере из датасета.

idx = 5
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
# print('label by model is', sentRuBert(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentRuBert(df_train.iloc[idx]['text'])[0]['score'])
print(f'label by model is {sentRuBert(df_train.iloc[idx]["text"])[0]["label"]} with score {round(sentRuBert(df_train.iloc[idx]["text"])[0]["score"], 5)}')

Манчестер через час играет, а я не дома (
label is 0
label by model is LABEL_0 with score 0.99585


Препроцессинг.

In [14]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [16]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [17]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [18]:
class BertClass(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        # self.bert = model_RuBert.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self.bert = BertModel.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        # print()
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

### Предсказание на валидационной выборке

In [20]:
model = BertClass().to(device)
# print(model)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClass(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [22]:
%%time
model.eval()

list_y_true = []
list_y_pred = []

test_acc, test_tp, test_fp, test_tn, test_fn = 0, 0, 0, 0, 0

for val_input, val_label in valid_loader:
    list_y_true += val_label
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask)

    list_y_pred += output.argmax(dim=1)

    test_acc += (output.argmax(dim=1) == val_label).sum().item()
    test_tp += ((val_label == 1) & (output.argmax(dim=1) == 1)).sum().item()
    test_tn += ((val_label == 0) & (output.argmax(dim=1) == 0)).sum().item()
    test_fp += ((val_label == 0) & (output.argmax(dim=1) == 1)).sum().item()
    test_fn += ((val_label == 1) & (output.argmax(dim=1) == 0)).sum().item()

test_precision = test_tp / (test_tp + test_fp) if (test_tp + test_fp) != 0 else 0
test_recall = test_tp / (test_tp + test_fn) if (test_tp + test_fn) != 0 else 0
test_acc = test_acc / len(df_val)


test_f1_score = 2 * test_precision * test_recall / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0

print(f'test accuracy = {test_acc: .3f}, test_precision = {test_precision: .3f}, test_recall = {test_recall: .3f}, test F1 score = {test_f1_score: .3f}')

test accuracy =  0.529, test_precision =  0.549, test_recall =  0.375, test F1 score =  0.446
CPU times: user 12.2 s, sys: 985 ms, total: 13.2 s
Wall time: 15.9 s


In [23]:
list_y_true = [list_y_true[i].item() for i in range(len(list_y_true))]
list_y_pred = [list_y_pred[i].item() for i in range(len(list_y_pred))]
print(classification_report(y_true=list_y_true, y_pred=list_y_pred))

              precision    recall  f1-score   support

           0       0.52      0.69      0.59     11234
           1       0.55      0.37      0.45     11449

    accuracy                           0.53     22683
   macro avg       0.53      0.53      0.52     22683
weighted avg       0.53      0.53      0.52     22683



### Дообучение модели

In [24]:
for epoch_num in range(3):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()

    list_y_true = []
    list_y_pred = []
    
    total_loss_val, total_acc_val = 0.0, 0.0
    test_acc, test_tp, test_fp, test_tn, test_fn = 0, 0, 0, 0, 0

    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        list_y_pred += output.argmax(dim=1)

        test_acc += (output.argmax(dim=1) == val_label).sum().item()
        test_tp += ((val_label == 1) & (output.argmax(dim=1) == 1)).sum().item()
        test_tn += ((val_label == 0) & (output.argmax(dim=1) == 0)).sum().item()
        test_fp += ((val_label == 0) & (output.argmax(dim=1) == 1)).sum().item()
        test_fn += ((val_label == 1) & (output.argmax(dim=1) == 0)).sum().item()

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
    
    test_precision = test_tp / (test_tp + test_fp) if (test_tp + test_fp) != 0 else 0
    test_recall = test_tp / (test_tp + test_fn) if (test_tp + test_fn) != 0 else 0
    test_acc = test_acc / len(df_val)

    test_f1_score = 2 * test_precision * test_recall / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')
    
    print(f'test accuracy = {test_acc: .3f}, test_precision = {test_precision: .3f}, test_recall = {test_recall: .3f}, test F1 score = {test_f1_score: .3f}')

100%|██████████| 2836/2836 [05:09<00:00,  9.16it/s]


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.577         | Val Loss:  0.010         | Val Accuracy:  0.590
test accuracy =  0.590, test_precision =  0.583, test_recall =  0.660, test F1 score =  0.619


100%|██████████| 2836/2836 [05:08<00:00,  9.19it/s]


Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.583         | Val Loss:  0.010         | Val Accuracy:  0.602
test accuracy =  0.602, test_precision =  0.587, test_recall =  0.714, test F1 score =  0.644


100%|██████████| 2836/2836 [05:09<00:00,  9.18it/s]


Epochs: 3 | Train Loss:  0.011         | Train Accuracy:  0.584         | Val Loss:  0.010         | Val Accuracy:  0.601
test accuracy =  0.601, test_precision =  0.584, test_recall =  0.728, test F1 score =  0.648


После дообучения видим рост метрик. Впрочем уже на 3 эпохе можно видеть, что accuracy перестаёт расти. И F1 score также замедляет рост.